# 3. Boreal Forest Height: Prediction
This notebook is to apply pre-trained Random Forest model to estimate potential forest height given environmental covariates

Based on Notebook by Zach Williams
Revision: JL 04/17/2023.
Revision: MJF 07/17/2023

Melanie Frost
melanie.frost@nasa.gov

### Deriving ensemble future canopy height and change predictions
We assembled the individual future potential canopy height predictions available from each GCM for each SSP and time period into ensemble predictions of future canopy height. To do this, we computed gridded maps of the median value of all individual GCM predictions for each SSP and future time period.  This calculation operated pixel-wise, based on the set of individual canopy height predictions from the available SSP- and time-period-specific GCMs summarized in Supplementary Figure 4. Then, we calculated maps of the differences between these future ensemble predictions and the current predictions. This step served to normalize the future predictions of potential canopy height with the same model-based predictions of current canopy height. This normalization was included to mitigate biases inherent in the model-based predictions of height. This produced 16 ensemble-based difference maps representing the potential differences in the future mean canopy height of vegetation for the 4 time-periods across the gradient of forest in our study domain according to the 4 CMIP6 SSPs. 

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import sys

import pandas as pd
import numpy as np
import treelite
import rioxarray as rxr
import rasterio as rio
import matplotlib
import geopandas as gpd

from osgeo import gdal,gdal_array,osr

import matplotlib.pyplot as plt

import panel as pn

from tqdm import tqdm

import holoviews as hv
import geoviews as gv
gv.extension('bokeh')
#gv.extension('matplotlib')

import xarray as xr

import joblib

#save the plots for comparison
import cartopy.crs as ccrs
from bokeh.resources import INLINE

import warnings
warnings.filterwarnings("ignore")


KeyboardInterrupt



In [ ]:
import glob
gv.__version__

In [ ]:
## Define global variables

#Circumpolar
#minlon = -179.02
#maxlon = 180
#minlat = 49.99
#maxlat = 74.95

#Eurasia
#minlon = 25
#maxlon = 179.02
#minlat = 52.01
#maxlat = 75.00


#04/17/2023
minlon = -168.98
maxlon = -50.02
minlat = 45.02
maxlat = 75

bbox = [minlon, minlat, maxlon, maxlat]

fn = '/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6/wc2.1_2.5m_bio/wc2.1_2.5m_bio_1.tif'
tmp = rxr.open_rasterio(fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat)
lons = tmp.x.values
lats = tmp.y.values
shape = (len(lats), len(lons))
print('Expected size:')
print(shape)
tmp=None

print('Lons:')
print(len(lons))
with rio.open(fn) as src:
    meta = src.meta
#print(meta)
src.close()
# color map for prediction
## 'black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'
clrs = ['#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850']
cmap_p = matplotlib.colors.LinearSegmentedColormap.from_list(name="color_p", 
                                                             colors=clrs, N=16) 

# color map for trend
brbg= matplotlib.cm.get_cmap('BrBG', 256)
grey = matplotlib.cm.get_cmap('Greys', 20)
gs = grey(np.linspace(0, 1, 20))
newcolors = brbg(np.linspace(0, 1, 256))
newcolors[85:170, :] = gs[5]
cmap_t = matplotlib.colors.ListedColormap(newcolors)

# set basemap
baseMap = gv.tile_sources.CartoLight
#baseMapRGB = gv.util.get_tile_rgb(baseMap,bbox=(minlon, minlat, maxlon, maxlat), zoom_level=1).opts(width=800, height=400, projection=ccrs.PlateCarree())

In [ ]:
# function to aggregate predictors
def build_predictors(bio_file, bnds, pf_fn=None, sg_fn=None, oc_fn=None):
    #get permafrost
    #Circumpolar
    #pf_fn = "/efs/zwwillia/BorealHeight_Data/Global_data/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_clip_global_25min.tif"
    #Eurasia
    #pf_fn = "/efs/zwwillia/BorealHeight_Data/Eurasia_data/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_clip_eurasia_25min_ext.tif"
    #NA
    #pf_fn = "/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_clip_25min.tif"
    try:
        pf = rxr.open_rasterio(pf_fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).sel(band=1).values.ravel()
        pf = pf.reshape(-1, 1)
        print('Perma shape')
        print(pf.shape)
    except:
        print('No Permafrost Added')
    
    try:
        sg = rxr.open_rasterio(sg_fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).sel(band=1).values.ravel()
        sg = sg.reshape(-1, 1)
        print('Depth to Bedrock shape')
        print(sg.shape)
    except:
        print('No Depth to Bedrock Added')
    
    try:
        oc = rxr.open_rasterio(oc_fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).sel(band=1).values.ravel()
        oc = oc.reshape(-1, 1)
        print('Organic Carbon Density shape')
        print(oc.shape)
    except:
        print('No Carbon Density Added')
    
    #get bioclim
    bios = extract_bioclim(bio_file, bnds)
    print('Bios Shape')
    print(bios.shape)
    x = np.concatenate([pf, sg, oc, bios], axis=1)
    mask = np.isnan(x).any(axis=1)
    
    return x, mask

# function to clip bioclim to aoi
def extract_bioclim(files, bnds):
    # function to read, clip & reshape bioclim 2.5m data
    if len(files)==19:
        sl = [rxr.open_rasterio(file, mask_and_scale=True).rio.clip_box(bnds[0], bnds[1], bnds[2], bnds[3]).values.flatten() for file in files]
        sub = np.concatenate([sl], axis=1)
        sub = np.moveaxis(sub, 0, -1)
    else:
        ds = rxr.open_rasterio(files[0], mask_and_scale=True).rio.clip_box(bnds[0], bnds[1], bnds[2], bnds[3])
        sub = np.moveaxis(ds.values, 0, -1).reshape((-1, 19))
    return sub
    
# funtion to plot result 
def map_bioclim(arr, mode="pred"):
    if mode == "pred":
        cmap = cmap_p
        clim = (0.5, 25.5)
        logz = True
    if mode == "diff":
        cmap = "BrBG"
        clim = (-5.0, 5.0)
        logz = False
    if mode == "trend":
        cmap = cmap_t
        clim = (-5.0, 5.0)
        logz = False
        
    img_opts = dict(
        width=600, 
        height=300, 
        logz=logz,
        cmap=cmap,
        colorbar=True,
        clim = clim,
        tools=["hover"], active_tools=['wheel_zoom']
        )
    gv_dataset = gv_dataset = gv.Dataset((lons, lats, arr), ['longitude', 'latitude'], mode)
    return gv.Image(gv_dataset).opts(**img_opts)


In [ ]:
import holoviews.operation.datashader as hd
# function to plot rasterized atl08 data  
def map_atl_raster(fn):
    ds = rxr.open_rasterio(fn, mask_and_scale=True)
    dt = ds.data.squeeze()
    #dt[mask2d] = np.nan

    gv_dataset = gv.Dataset((ds.x.values, ds.y.values, dt), ['lon', 'lat'], 'h_can')
    cls = ['black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850']
    color = matplotlib.colors.LinearSegmentedColormap.from_list(name='myc', colors=cls)

    base_map =  hv.element.tiles.EsriImagery()
    img = gv.Image(gv_dataset).opts(
            height=600, width=1100, colorbar=True, active_tools=['wheel_zoom'],
            tools=['hover'], cmap=color, clim=(0.0, 25.0))
    return hd.regrid(img)

In [ ]:
boreal_ext = gpd.read_file('/explore/nobackup/people/mfrost2/projects/boreal_hcan/GIS/NA_boreal_ext_buff.shp')
#boreal_ext.plot()
fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_25min.tif"
xds = rxr.open_rasterio(fn, mask_and_scale=True)
clipped = xds.rio.clip(boreal_ext.geometry.values, boreal_ext.crs, drop=False, invert=False)
#clipped.plot()

## Load pretrained model

In [ ]:
rf_model = joblib.load('/explore/nobackup/people/mfrost2/projects/boreal_hcan/models/rf_NA_ATL_max_Perm_Soil.joblib')

## Apply model on bioclim current

In [ ]:
%%time
pf_fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_25min.tif"
sg_fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/BDRICM_M_250m_ll_clip_25min.tif"
oc_fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/OCDENS_M_sl1_250m_ll_clip_25min.tif"

flist = [f"/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6/wc2.1_2.5m_bio/wc2.1_2.5m_bio_{id}.tif" for id in range(1,20)]
xc, maskc = build_predictors(flist, bbox, pf_fn=pf_fn, sg_fn=sg_fn, oc_fn=oc_fn)

In [ ]:
res = rf_model.predict(xc)

In [ ]:
# Fill back nodata & reshape back to 2D
res[maskc]=np.nan
predc = res.reshape(shape)

In [ ]:
# Show prediction
img0 = map_bioclim(predc, 'pred')
pn.Column(img0*baseMap)

In [ ]:
# Save output arrays as GeoTiffs
# Blueprint is target
def save_map(arr,meta,outPut):
    #using GDAL
    nrows,ncols = np.shape(arr)
    x_res = (maxlon-minlon)/float(ncols)
    y_res = (maxlat-minlat)/float(nrows)
    geotransform = (minlon,x_res,0,maxlat,0,-y_res)
    output_raster = gdal.GetDriverByName('GTiff').Create(outPut,ncols,nrows,1,gdal.GDT_Float32)
    output_raster.SetGeoTransform(geotransform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    output_raster.SetProjection(srs.ExportToWkt())
    #output_raster.rxr.clip(boreal_ext.geometry.apply(mapping),boreal_ext.crs)
    output_raster.GetRasterBand(1).WriteArray(arr)
    output_raster.FlushCache()
    print('Raster saved!')

In [ ]:
# Save current map for future analysis
# save_map(predc,meta,'/explore/nobackup/people/mfrost2/projects/boreal_hcan/output/North_America_Current_filtered_noPerm_20230706.tif')
# current_pred = gpd.read_file('/explore/nobackup/people/mfrost2/projects/boreal_hcan/output/North_America_Current_filtered_noPerm_20230706.tif')

## Apply model on bioclim future (CMIP6)
ref: https://www.worldclim.org/data/cmip6/cmip6_clim2.5m.html

In [ ]:
# Define variables
cmip_path = "/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6"

md_list_2021 = ['BCC-CSM2-MR','CanESM5','CNRM-CM6-1','MIROC-ES2L','MIROC6', 'CNRM-ESM2-1']
md_list_2041 = ['CanESM5','CNRM-CM6-1','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MRI-ESM2-0', 'CNRM-ESM2-1']
md_list_2061 = ['BCC-CSM2-MR','CanESM5','CNRM-CM6-1','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MRI-ESM2-0', 'CNRM-ESM2-1']
md_list_2081 = ['BCC-CSM2-MR','CanESM5','CNRM-CM6-1','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MRI-ESM2-0', 'CNRM-ESM2-1']

md_list = md_list_2061

sn = ['ssp126','ssp245','ssp370','ssp585']
#yr = '2021-2040'
#yr = '2041-2060'
yr = '2061-2080'
#yr = '2081-2100'

In [ ]:
# Apply model on each CMIP-6 model and year commbination
preds = []
diffs = []

#label for organizing outputs
reg = 'NorthAmerica'
outDir = r'/explore/nobackup/people/mfrost2/projects/boreal_hcan/output/'
pf_fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_25min.tif"
with tqdm(total=len(sn)) as pbar:
    for s in sn:
        for md in md_list:
            fl = os.path.join(cmip_path, f"wc2.1_2.5m_bioc_{md}_{s}_{yr}.tif")
            pbar.write(f'processing: {os.path.basename(fl)}')
            pbar.update(1)
            x, msk= build_predictors([fl], bbox, pf_fn=pf_fn, sg_fn=sg_fn, oc_fn=oc_fn)        
            #if permafrost/soils removed
            #x = x[:,2:]
            #res = treelite.gtil.predict(tl_model, x, pred_margin=True)
            res = rf_model.predict(x, predict_model='GPU')
            res[msk]=np.nan
            pred = res.reshape(shape)
            #save to file
        
            preds.append(pred)
            diff = pred-predc
            diffs.append(diff)
            outfile_pred = os.path.join(outDir, f"{reg}_{md}_{s}_{yr}.tif")
            outfile_diff = os.path.join(outDir, f"{reg}_{md}_{s}_{yr}_DIFF.tif")
            #save_map(pred,meta,outfile_pred)
            #save_map(diff,meta,outfile_diff)

In [ ]:
res

In [ ]:
# Put maps in order
rows = []
for i in range(0, len(sn)):
    pimg = map_bioclim(preds[i], 'pred')
    title = f"Predicted Canopy Height {yr} CMIP6 {sn[i]} (m)"
    pimg.opts(title=title)
    dimg = map_bioclim(diffs[i], 'diff')
    title = f"Predicted Canopy Height CMIP6 {sn[i]} vs. Current (m)"
    dimg.opts(title=title)
    rows.append(pimg*baseMap+dimg*baseMap)

In [ ]:
#test = gpd.clip(preds[0], boreal_ext)

In [ ]:
rows

In [ ]:
# Show maps
pn.Column(rows[0], 
          rows[1],
          rows[2],
          rows[3])

# Scatter plot among scenarios

In [ ]:
# Define a pair of predictions, e.g. ('ssp126', 'ssp245') or ('current', 'ssp370')
pairs = ('current', 'ssp585')


In [ ]:
pred_dict = {'current' : predc,
            'ssp126' : preds[0],
            'ssp245': preds[1],
            'ssp370': preds[1],
            'ssp585': preds[2]}

In [ ]:
p1 = pred_dict[pairs[0]].ravel()
p1 = p1[~np.isnan(p1)]
p2 = pred_dict[pairs[1]].ravel()
p2 = p2[~np.isnan(p2)]
data = np.column_stack((p1, p2))
sample_size = 2000
s = np.random.choice(data.shape[0], sample_size, replace=False)
sample = data[s, :]
pts = hv.Points(sample).opts(color='k', marker='+', size=10)
pn.Column(pts.opts(width=800, height=400, xlabel=f"{pairs[0].upper()} Predicted h-can", ylabel=f"{pairs[1].upper()} Predicted h-can"))

### Medians

In [ ]:
# For 2021

md_list = md_list_2021
yr = '2021-2040'

j = 0
for i in sn:
    median_map = np.median([preds[j + 0],preds[j + 1],preds[j + 2],preds[j + 3],preds[j + 4],preds[j + 5]],axis=0)
    print(median_map.shape)
    median_diff = median_map-predc
    
    outfile_pred = os.path.join(outDir, f"Median_{i}_{yr}.tif")
    outfile_diff = os.path.join(outDir, f"Median_{i}_{yr}_DIFF.tif")
    print(outfile_pred)
    save_map(median_map,meta,outfile_pred)
    save_map(median_diff,meta,outfile_diff)
    
    j = j + 6

In [ ]:
# For 2041

md_list = md_list_2041
yr = '2041-2060'

j = 0
for i in sn:
    
    median_map = np.median([preds[j + 0],preds[j + 1],preds[j + 2],preds[j + 3],preds[j + 4],preds[j + 5],preds[j + 6]], axis=0)
    
    print(median_map.shape)
    median_diff = median_map-predc
    
    outfile_pred = os.path.join(outDir, f"Median_{i}_{yr}.tif")
    outfile_diff = os.path.join(outDir, f"Median_{i}_{yr}_DIFF.tif")
    print(outfile_pred)
    save_map(median_map,meta,outfile_pred)
    save_map(median_diff,meta,outfile_diff)
    
    j = j + 7

In [ ]:
# For 2061
j = 0

md_list = md_list_2061
yr = '2061-2080'

for i in sn:
    
    median_map = np.median([preds[j + 0],preds[j + 1],preds[j + 2],preds[j + 3],preds[j + 4],preds[j + 5],preds[j + 6],preds[j + 7]],axis=0)
    
    print(median_map.shape)
    median_diff = median_map-predc
    
    outfile_pred = os.path.join(outDir, f"Median_{i}_{yr}.tif")
    outfile_diff = os.path.join(outDir, f"Median_{i}_{yr}_DIFF.tif")
    print(outfile_pred)
    save_map(median_map,meta,outfile_pred)
    save_map(median_diff,meta,outfile_diff)
    
    j = j + 8

In [ ]:
# For 2081
j = 0

md_list = md_list_2081
yr = '2081-2100'

for i in sn:
    
    median_map = np.median([preds[j + 0],preds[j + 1],preds[j + 2],preds[j + 3],preds[j + 4],preds[j + 5],preds[j + 6],preds[j + 7]],axis=0)
    
    print(median_map.shape)
    median_diff = median_map-predc
    
    outfile_pred = os.path.join(outDir, f"Median_{i}_{yr}.tif")
    outfile_diff = os.path.join(outDir, f"Median_{i}_{yr}_DIFF.tif")
    print(outfile_pred)
    save_map(median_map,meta,outfile_pred)
    save_map(median_diff,meta,outfile_diff)
    
    j = j + 8

In [ ]:
pimg = map_bioclim(median_map, 'pred')
title = f"Median Predicted HCan {yr} {sn[int(j/7-1)]}"
pimg.opts(title=title)

In [ ]:
plt.close('all')

## Clip Rasters to Study Domain

In [ ]:
# Clip Rasters to NA Boreal Extent

boreal_ext = gpd.read_file('/explore/nobackup/people/mfrost2/projects/boreal_hcan/GIS/NA_boreal_ext_buff.shp')
dp = r'/explore/nobackup/people/mfrost2/projects/boreal_hcan/output/'

med_fn = 'Median*0.tif'
diff_fn = 'Median*DIFF.tif'

med_path = os.path.join(dp, med_fn)
diff_path = os.path.join(dp, diff_fn)

med_list = glob.glob(med_path)
diff_list = glob.glob(diff_path)

diff_list

In [ ]:
# Make clipped rasters of median values

#for raster in diff_list:
for raster in med_list:
    xds = rxr.open_rasterio(raster, mask_and_scale=True)
    clipped = xds.rio.clip(boreal_ext.geometry.values, boreal_ext.crs, drop=False, invert=False)
    
    new_fp = str(raster)[:-4]+'_clipped.tif'
    print(new_fp)
    #clipped.rio.to_raster(new_fp)

In [ ]:
# See if clip worked
clipped.plot()

In [ ]:
#Check spatial resolution
# Hcan output file
import rasterio as rio

raster = gdal.Open('/explore/nobackup/people/mfrost2/projects/boreal_hcan/output/Median_ssp126_2061-2080.tif' )

gdal.Info(raster)
raster.GetGeoTransform()

In [ ]:
#Bioclim output file
raster = gdal.Open(flist[0])

gdal.Info(raster)
raster.GetGeoTransform()

In [ ]:
#ATL Input file
atl_file = '/explore/nobackup/projects/ilab/projects/forest_height/boreal/atl08/atl08_boreal_na_20m_intersect_filt_2018_2022_max_raster.tif'
raster = gdal.Open(atl_file)

gdal.Info(raster)
raster.GetGeoTransform()

In [ ]:
#Bioclim input file
raster = gdal.Open(flist[0])

gdal.Info(raster)
raster.GetGeoTransform()

In [ ]:
bio_path = "/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim/wc2.0_30s_bio"
bio_idx = [str(x).zfill(2) for x in range(1,20)]
bio_files = [os.path.join(bio_path, f"wc2.0_bio_30s_{i}.tif") for i in bio_idx]
bio_files